In [1]:
import requests
import re

In [111]:
accepted_units = ['small', 'medium', 'big', 'medium-size',
                  'g', 'pound', 'pounds', 
                  'ounce', 'ounces', 'l', 'ml',
                  'jar', 'cup', 'cups', 'can',
                  'spoon', 'tablespoon',
                  'pinch', 'handful',
                  'cloves'
                 ]

In [112]:
top_source = 'https://www.cookingchanneltv.com/recipes/a-z/p/p/3'
source_letter = 'P'

In [3]:
r = requests.get(top_source)

In [4]:
# r.content

In [5]:
content = str(r.content)

In [6]:
idx_start = content.find('<span class="o-Capsule__a-HeadlineText">'+source_letter+'</span>')
content = content[idx_start:]
idx_stop = content.find('<section class="o-Pagination ">')
content = content[:idx_stop]

In [7]:
recepies_url_list = []
for ln in content.split('href')[1:]:
    idx_end = ln.find('">')
    url = 'https:' + ln[2:idx_end]
    recepies_url_list.append(url)

In [8]:
url = recepies_url_list[3]
print(url)
content = str(requests.get(url).content)

https://www.cookingchanneltv.com/recipes/debi-mazar-and-gabriele-corcos/pasta-alla-gricia-2105351


In [15]:
def getName(content):
    template = '<span class="o-AssetTitle__a-HeadlineText">'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:idx_start+100]
    idx_stop = s.find('</span>')
    title = s[:idx_stop]
    if ':' in title:
        title = title[:title.find(':')]
    print('\t'+title)

In [16]:
def getLevel(content):
    template = '<span class="o-RecipeInfo__a-Headline">Level:</span>'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:idx_start+100]
    out = re.search(r'">[A-Za-z]+</span', s)
    level = out.group(0)[2:-6]
    print('\t'+level)

In [17]:
def getNumberPeople(content):
    template = '<span class="o-RecipeInfo__a-Headline">Yield:</span>'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:idx_start+100]
    out = re.search(r'">[0-9]+', s)
    Nppl = out.group(0)[2:]
    print('\t'+Nppl)

In [117]:
def getIngredients(content):
    template = '<p class="o-Ingredients__a-Ingredient">'
    idx_start = content.find(template) + len(template)
    s = content[idx_start:]
    # print(s)
    cut_template = '<h6 class="o-Ingredients__a-SubHeadline">'
    if cut_template in s:
        idx_stop = s.find(cut_template)
        s = s[:idx_stop]
    slist = s.split(template)
    # print(slist)
    for l in slist:
        i = l.find('</p>')
        aux = l[:i]
#         print(aux)
        if ',' in aux:
            aux = aux[:aux.find(',')]
        if '(' in aux:
            aux = aux[:aux.find('(')-1] + aux[aux.find(')')+1:]
        if '/' in aux:
            aux = aux[aux.find('/')+1:]
        if aux[-8:] == '\\xc2\\xa0':
            aux = aux[:-8]

        l_aux = aux.split(' ')        
        if len(l_aux)==2 and l_aux[0].isnumeric():
            name = l_aux[1].capitalize()
            amount = l_aux[0]
            units = ''
        elif len(l_aux) > 2 and (l_aux[1] in accepted_units):
            name = ' '.join(l_aux[2:])
            name.capitalize()
            amount = l_aux[0]
            units = l_aux[1]
        elif len(l_aux) > 1 and not l_aux[0].isnumeric() and not (l_aux[1] in accepted_units):
            name = ' '.join(l_aux)
            name.capitalize()
            amount = ''
            units = ''
        elif len(l_aux) > 2 and l_aux[0].isnumeric() and not (l_aux[1] in accepted_units):
            name = ' '.join(l_aux[1:])
            name.capitalize()
            amount = l_aux[0]
            units = ''
        elif len(l_aux) == 1:
            name = l_aux[0]
            amount = ''
            units = ''
        else:
            print(l_aux)
            name = 'NaN'
            amount = 'NaN'
            units = 'NaN'
        
        out = '\t\t'+ name + '-->'
        out += 'Amount: '+ amount + '(' + units + ')'
        print(out)# + '\t\t~~  '+l[:i])
    
#     print('\n\n\n')

In [118]:
# content

In [120]:
for url in recepies_url_list[5:10]:
    print(url)
    content = str(requests.get(url).content)
    getName(content)
    getLevel(content)
    getNumberPeople(content)
    getIngredients(content)
    print()

https://www.cookingchanneltv.com/recipes/david-rocco/pasta-anto-te-1961156
	Pasta Anto-Te
	Easy
	4
		G-->Amount: 228()
		cup/117 ml milk-->Amount: 2()
		unsalted butter-->Amount: 14(g)
		fresh tagliolini pasta-->Amount: 362(g)
		extra-virgin olive oil-->Amount: 59(ml)
		rosewater-->Amount: 44(ml)
		cup/114 g pistachios-->Amount: 2()
		ounces/100 g fresh sheep\'s milk ricotta cheese-->Amount: 2()
		Sprinkle saffron-->Amount: ()
		Salt-->Amount: ()
		Bunch fresh flat-leaf parsley-->Amount: ()

https://www.cookingchanneltv.com/recipes/tyler-florence/pasta-dough-for-ravioli-1943504
	Pasta Dough for Ravioli
	Intermediate


AttributeError: 'NoneType' object has no attribute 'group'